<a href="https://colab.research.google.com/github/ushasri999/Multiclass-classification-of-DDoS-attacks-in-IoT-network-using-hybrid-feature-selection-algorithm/blob/main/RF_34.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**34 classes, 15 features**

In [ ]:
#5
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_selection import SelectKBest, f_classif, RFE
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Step 1: Load the filtered dataset
df = pd.read_csv('/content/drive/MyDrive/major project/data.csv')  # Load your actual dataset here

# Step 2: Separate features and target
X = df.drop("label", axis=1, errors='ignore')  # Avoid crashing if 'label' does not exist
y = df["label"] if 'label' in df.columns else pd.Series([])  # Handle case where label is missing

# Check if X and y are valid
print(X.shape)  # Should not be empty
print(y.shape)  # Should not be empty

# Step 3: Handle categorical and missing data
X = pd.get_dummies(X, drop_first=True)  # One-hot encode categorical features
X.fillna(X.mean(), inplace=True)  # Fill missing values with the mean

(238687, 46)
(238687,)


In [ ]:
# Step 4: Hybrid Feature Selection
filter_selector = SelectKBest(score_func=f_classif, k=8)  # Select only top 5 features using filter method
X_new_filter = filter_selector.fit_transform(X, y)
filter_scores = filter_selector.scores_

# Wrapper Method: Recursive Feature Elimination (RFE) with Random Forest
rfe_selector = RFE(estimator=RandomForestClassifier(), n_features_to_select=7)  # Select 5 features
X_new_wrapper = rfe_selector.fit_transform(X, y)
rfe_ranking = rfe_selector.ranking_

# Combine results: Select top 5 features from both methods
top_features_filter = np.argsort(filter_scores)[-8:]  # Top 5 features from filter method
print('top_features_filter ', top_features_filter)
top_features_rfe = np.where(rfe_ranking == 1)[0]  # Features selected by RFE
print('top_features_rfe ', top_features_rfe)
top_features_combined = list(set(top_features_filter) | set(top_features_rfe))  # Combine both selections
print('top_features_combined ', top_features_combined)

# Step 5: Select top features from the dataset
X_selected = X.iloc[:, top_features_combined]

# Print the names of the selected features
print("Selected Features:", X_selected.columns.tolist())


/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [12 13 22 23 25 28] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


top_features_filter  [30  7 28 23 13 12 25 22]
top_features_rfe  [ 1  8 34 36 38 39 41]
top_features_combined  [1, 34, 36, 38, 7, 39, 8, 41, 12, 13, 22, 23, 25, 28, 30]
Selected Features: ['Header_Length', 'Min', 'AVG', 'Tot size', 'fin_flag_number', 'IAT', 'syn_flag_number', 'Magnitue', 'ece_flag_number', 'cwr_flag_number', 'Telnet', 'SMTP', 'IRC', 'DHCP', 'ICMP']


In [ ]:
# Step 6: Train and Test Split
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.3, random_state=42)

# Step 7: Train Random Forest Classifier
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

# Step 8: Evaluate the model
y_pred = rf_model.predict(X_test)
# Metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')  # Weighted for imbalanced classes
recall = recall_score(y_test, y_pred, average='weighted')  # Weighted for imbalanced classes
f1 = f1_score(y_test, y_pred, average='weighted')  # Weighted for imbalanced classes

# Print metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Classification Report:\n", classification_report(y_test, y_pred))

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", conf_matrix)

# Step 9: Cross-validation (optional)
cv_scores = cross_val_score(rf_model, X_selected, y, cv=5)
print("Cross-validation Accuracy:", np.mean(cv_scores))

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.990224419400338
Precision: 0.9895425589448519
Recall: 0.990224419400338
F1 Score: 0.9894958547608352


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Classification Report:
                          precision    recall  f1-score   support

       Backdoor_Malware       0.00      0.00      0.00         4
          BenignTraffic       0.82      0.97      0.88      1667
       BrowserHijacking       0.00      0.00      0.00         8
       CommandInjection       0.00      0.00      0.00         6
 DDoS-ACK_Fragmentation       0.99      0.98      0.99       463
        DDoS-HTTP_Flood       0.73      0.90      0.81        52
        DDoS-ICMP_Flood       1.00      1.00      1.00     11141
DDoS-ICMP_Fragmentation       0.98      0.99      0.99       685
      DDoS-PSHACK_Flood       1.00      1.00      1.00      6319
       DDoS-RSTFINFlood       1.00      1.00      1.00      6238
         DDoS-SYN_Flood       1.00      1.00      1.00      6261
         DDoS-SlowLoris       0.76      0.62      0.68        40
DDoS-SynonymousIP_Flood       1.00      1.00      1.00      5354
         DDoS-TCP_Flood       1.00      1.00      1.00      6929


**34 classes, 10 features**

In [ ]:
# Step 4: Hybrid Feature Selection
filter_selector = SelectKBest(score_func=f_classif, k=5)  # Select only top 5 features using filter method
X_new_filter = filter_selector.fit_transform(X, y)
filter_scores = filter_selector.scores_

# Wrapper Method: Recursive Feature Elimination (RFE) with Random Forest
rfe_selector = RFE(estimator=RandomForestClassifier(), n_features_to_select=5)  # Select 5 features
X_new_wrapper = rfe_selector.fit_transform(X, y)
rfe_ranking = rfe_selector.ranking_

# Combine results: Select top 5 features from both methods
top_features_filter = np.argsort(filter_scores)[-5:]  # Top 5 features from filter method
print('top_features_filter ', top_features_filter)
top_features_rfe = np.where(rfe_ranking == 1)[0]  # Features selected by RFE
print('top_features_rfe ', top_features_rfe)
top_features_combined = list(set(top_features_filter) | set(top_features_rfe))  # Combine both selections
print('top_features_combined ', top_features_combined)

# Step 5: Select top features from the dataset
X_selected = X.iloc[:, top_features_combined]

# Print the names of the selected features
print("Selected Features:", X_selected.columns.tolist())


/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [12 13 22 23 25 28] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


top_features_filter  [23 13 12 25 22]
top_features_rfe  [15 34 38 39 41]
top_features_combined  [34, 38, 39, 41, 12, 13, 15, 22, 23, 25]
Selected Features: ['Min', 'Tot size', 'IAT', 'Magnitue', 'ece_flag_number', 'cwr_flag_number', 'syn_count', 'Telnet', 'SMTP', 'IRC']


In [ ]:
# Step 6: Train and Test Split
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.3, random_state=42)

# Step 7: Train Random Forest Classifier
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

# Step 8: Evaluate the model
y_pred = rf_model.predict(X_test)
# Metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')  # Weighted for imbalanced classes
recall = recall_score(y_test, y_pred, average='weighted')  # Weighted for imbalanced classes
f1 = f1_score(y_test, y_pred, average='weighted')  # Weighted for imbalanced classes

# Print metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Classification Report:\n", classification_report(y_test, y_pred))

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", conf_matrix)

# Step 9: Cross-validation (optional)
cv_scores = cross_val_score(rf_model, X_selected, y, cv=5)
print("Cross-validation Accuracy:", np.mean(cv_scores))

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.99111818676945
Precision: 0.9908117867962567
Recall: 0.99111818676945
F1 Score: 0.9904936878357297


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Classification Report:
                          precision    recall  f1-score   support

       Backdoor_Malware       0.00      0.00      0.00         4
          BenignTraffic       0.82      0.96      0.88      1667
       BrowserHijacking       1.00      0.12      0.22         8
       CommandInjection       0.00      0.00      0.00         6
 DDoS-ACK_Fragmentation       0.98      0.98      0.98       463
        DDoS-HTTP_Flood       0.84      0.98      0.90        52
        DDoS-ICMP_Flood       1.00      1.00      1.00     11141
DDoS-ICMP_Fragmentation       0.99      1.00      0.99       685
      DDoS-PSHACK_Flood       1.00      1.00      1.00      6319
       DDoS-RSTFINFlood       1.00      1.00      1.00      6238
         DDoS-SYN_Flood       1.00      1.00      1.00      6261
         DDoS-SlowLoris       0.94      0.85      0.89        40
DDoS-SynonymousIP_Flood       1.00      1.00      1.00      5354
         DDoS-TCP_Flood       1.00      1.00      1.00      6929


**34 classes, 5 features**

In [ ]:
# Step 4: Hybrid Feature Selection
filter_selector = SelectKBest(score_func=f_classif, k=3)  # Select only top 5 features using filter method
X_new_filter = filter_selector.fit_transform(X, y)
filter_scores = filter_selector.scores_

# Wrapper Method: Recursive Feature Elimination (RFE) with Random Forest
rfe_selector = RFE(estimator=RandomForestClassifier(), n_features_to_select=2)  # Select 5 features
X_new_wrapper = rfe_selector.fit_transform(X, y)
rfe_ranking = rfe_selector.ranking_

# Combine results: Select top 5 features from both methods
top_features_filter = np.argsort(filter_scores)[-3:]  # Top 5 features from filter method
print('top_features_filter ', top_features_filter)
top_features_rfe = np.where(rfe_ranking == 1)[0]  # Features selected by RFE
print('top_features_rfe ', top_features_rfe)
top_features_combined = list(set(top_features_filter) | set(top_features_rfe))  # Combine both selections
print('top_features_combined ', top_features_combined)

# Step 5: Select top features from the dataset
X_selected = X.iloc[:, top_features_combined]

# Print the names of the selected features
print("Selected Features:", X_selected.columns.tolist())


/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [12 13 22 23 25 28] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


top_features_filter  [12 25 22]
top_features_rfe  [39 41]
top_features_combined  [22, 39, 25, 12, 41]
Selected Features: ['Telnet', 'IAT', 'IRC', 'ece_flag_number', 'Magnitue']


In [ ]:
# Step 6: Train and Test Split
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.3, random_state=42)

# Step 7: Train Random Forest Classifier
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

# Step 8: Evaluate the model
y_pred = rf_model.predict(X_test)
# Metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')  # Weighted for imbalanced classes
recall = recall_score(y_test, y_pred, average='weighted')  # Weighted for imbalanced classes
f1 = f1_score(y_test, y_pred, average='weighted')  # Weighted for imbalanced classes

# Print metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Classification Report:\n", classification_report(y_test, y_pred))

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", conf_matrix)

# Step 9: Cross-validation (optional)
cv_scores = cross_val_score(rf_model, X_selected, y, cv=5)
print("Cross-validation Accuracy:", np.mean(cv_scores))

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.990559582163755
Precision: 0.9903493089521479
Recall: 0.990559582163755
F1 Score: 0.9901937233497596


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Classification Report:
                          precision    recall  f1-score   support

       Backdoor_Malware       0.50      0.75      0.60         4
          BenignTraffic       0.81      0.93      0.87      1667
       BrowserHijacking       0.67      0.25      0.36         8
       CommandInjection       0.33      0.50      0.40         6
 DDoS-ACK_Fragmentation       0.99      0.99      0.99       463
        DDoS-HTTP_Flood       1.00      1.00      1.00        52
        DDoS-ICMP_Flood       1.00      1.00      1.00     11141
DDoS-ICMP_Fragmentation       1.00      1.00      1.00       685
      DDoS-PSHACK_Flood       1.00      1.00      1.00      6319
       DDoS-RSTFINFlood       1.00      1.00      1.00      6238
         DDoS-SYN_Flood       1.00      1.00      1.00      6261
         DDoS-SlowLoris       1.00      1.00      1.00        40
DDoS-SynonymousIP_Flood       1.00      1.00      1.00      5354
         DDoS-TCP_Flood       1.00      1.00      1.00      6929
